Research paper implementation

In [ ]:
import cv2
import numpy as np
from scipy.ndimage import laplace
import os
from matplotlib import pyplot as plt

def detect_edges(image, sigma=1.0, low_thresh=50, high_thresh=150):
    """Step 1: Detect edges using Marr-Hildreth operator."""
    blurred = cv2.GaussianBlur(image, (5, 5), sigma)
    laplacian = cv2.Laplacian(blurred, cv2.CV_64F)
    edges = cv2.Canny(np.uint8(laplacian), low_thresh, high_thresh)
    return edges

def quantize_and_subsample(image, edges, quantization_levels=16, subsample_step=5):
    """Step 2: Quantize and subsample pixel values near edges."""
    quantized = np.round(image / 255 * (quantization_levels - 1)) * (255 / (quantization_levels - 1))
    subsampled = np.zeros_like(image)
    edge_coords = np.argwhere(edges > 0)
    for i, (x, y) in enumerate(edge_coords):
        if i % subsample_step == 0:
            subsampled[x, y] = quantized[x, y]
    return subsampled

def homogeneous_diffusion(encoded_image, edges, iterations=100, alpha=0.1):
    """Step 3: Reconstruct missing values using homogeneous diffusion."""
    reconstructed = np.copy(encoded_image)
    edge_mask = edges > 0  # Fixed pixels are edges
    for _ in range(iterations):
        laplacian_update = laplace(reconstructed)
        reconstructed = reconstructed + alpha * laplacian_update
        reconstructed[edge_mask] = encoded_image[edge_mask]  # Keep edge values fixed
    return reconstructed

def calculate_psnr(original, reconstructed):
    """Calculate PSNR to compare image quality."""
    mse = np.mean((original - reconstructed) ** 2)
    if mse == 0:
        return float('inf')
    psnr = 10 * np.log10(255**2 / mse)
    return psnr

def process_images_from_folder(folder_path, output_folder, sigma=1.0, low_thresh=50, high_thresh=150, quantization_levels=16, subsample_step=5, iterations=100, alpha=0.1):
    """Process all images in the dataset folder."""
    os.makedirs(output_folder, exist_ok=True)
    
    # Use the image fetching method as described
    image_files = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(folder_path)) for f in fn if f.endswith(('.jpg', '.png', '.jpeg', '.bmp', '.tif', '.tiff'))]

    for image_path in image_files:
        # Load image
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        # Step 1: Detect edges
        edges = detect_edges(image, sigma, low_thresh, high_thresh)
        
        # Step 2: Quantize and subsample
        quantized_subsampled = quantize_and_subsample(image, edges, quantization_levels, subsample_step)
        
        # Step 3: Reconstruct using diffusion
        reconstructed_image = homogeneous_diffusion(quantized_subsampled, edges, iterations, alpha)
        
        # Calculate PSNR
        psnr_value = calculate_psnr(image, reconstructed_image)
#         print(f"Processed {image_path}: PSNR = {psnr_value:.2f} dB")
        
        # Save results
        cv2.imwrite(os.path.join(output_folder, f"{os.path.splitext(os.path.basename(image_path))[0]}_edges.png"), edges)
        cv2.imwrite(os.path.join(output_folder, f"{os.path.splitext(os.path.basename(image_path))[0]}_reconstructed.png"), reconstructed_image)
        
        # Plot and save comparison
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1), plt.title('Original'), plt.imshow(image, cmap='gray')
        plt.subplot(1, 3, 2), plt.title('Edges'), plt.imshow(edges, cmap='gray')
        plt.subplot(1, 3, 3), plt.title('Reconstructed'), plt.imshow(reconstructed_image, cmap='gray')
        plt.savefig(os.path.join(output_folder, f"{os.path.splitext(os.path.basename(image_path))[0]}_comparison.png"))
        plt.close()

# Paths
dataset_folder = r"OneDrive - Indian Institute of Technology Bombay/Documents/Data_science_intern/CS663/Weizmann_Seg_DB_1obj/1obj"  # Your dataset folder
output_folder = r"OneDrive - Indian Institute of Technology Bombay/Documents/Data_science_intern/CS663/Output"  # Replace with your desired output folder

# Process all images in the dataset folder
process_images_from_folder(dataset_folder, output_folder)


Research paper implementation & pca

In [ ]:
import cv2
import numpy as np
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
import os

# PCA-based Image Compression
def pca_image_compression(image, n_components=50):
    """Compress an image using PCA (Principal Component Analysis)."""
    height, width = image.shape
    n_components = min(n_components, min(height, width))  # Limit components to the smaller dimension

    # Perform PCA on the image along the width (columns)
    pca = PCA(n_components=n_components)
    transformed = pca.fit_transform(image)  # Transform the image using PCA
    reconstructed = pca.inverse_transform(transformed)  # Reconstruct the image

    return reconstructed

# Edge-based Compression Placeholder Functions
def extract_edges(image, sigma=1.0, low_thresh=50, high_thresh=150):
    """Extract edges using Canny edge detection."""
    blurred = cv2.GaussianBlur(image, (5, 5), sigma)
    edges = cv2.Canny(blurred, low_thresh, high_thresh)
    return edges

def reconstruct_image(original_image, edges):
    """Reconstruct the image using edges."""
    reconstructed = cv2.bitwise_and(original_image, original_image, mask=edges)
    return reconstructed

# PSNR Calculation
def calculate_psnr(original, reconstructed):
    """Calculate PSNR to compare image quality."""
    mse = np.mean((original - reconstructed) ** 2)
    if mse == 0:
        return float('inf')
    psnr = 10 * np.log10(255**2 / mse)
    return psnr

# Compare Edge-based and PCA-based Compression
def compare_compression_methods(image, n_components=50, sigma=1.0, low_thresh=50, high_thresh=150):
    """Compare edge-based compression with PCA-based compression."""
    # Edge-based Compression
    edges = extract_edges(image, sigma, low_thresh, high_thresh)
    reconstructed_edge_image = reconstruct_image(image, edges)
    psnr_edge = calculate_psnr(image, reconstructed_edge_image)
    
    # PCA-based Compression
    reconstructed_pca_image = pca_image_compression(image, n_components)
    psnr_pca = calculate_psnr(image, reconstructed_pca_image)
    
    print(f"Edge-based Compression PSNR: {psnr_edge:.2f} dB")
    print(f"PCA-based Compression PSNR: {psnr_pca:.2f} dB")
    
    # Plot and save comparison
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1), plt.title('Original'), plt.imshow(image, cmap='gray')
    plt.subplot(1, 3, 2), plt.title('Edge-based Reconstructed'), plt.imshow(reconstructed_edge_image, cmap='gray')
    plt.subplot(1, 3, 3), plt.title('PCA-based Reconstructed'), plt.imshow(reconstructed_pca_image, cmap='gray')
    plt.show()

# Process Images in Dataset with PCA Comparison
def process_images_and_compare(folder_path, output_folder, n_components=50, sigma=1.0, low_thresh=50, high_thresh=150):
    """Process all images in the dataset folder and compare both compression methods."""
    os.makedirs(output_folder, exist_ok=True)

    # Fetch all image files in the folder
    image_files = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(folder_path)) for f in fn if f.endswith(('.jpg', '.png', '.jpeg', '.bmp', '.tif', '.tiff'))]

    for image_path in image_files:
        # Load image
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        if image is None:
            print(f"Error loading image: {image_path}")
            continue

        # Save original image
        base_filename = os.path.splitext(os.path.basename(image_path))[0]
        cv2.imwrite(os.path.join(output_folder, f"{base_filename}_original.png"), image)

        # Compare the two compression methods
        compare_compression_methods(image, n_components, sigma, low_thresh, high_thresh)
        
        # Optionally save the images for the two methods as well

        # Edge-based Reconstructed Image
        edges = extract_edges(image, sigma, low_thresh, high_thresh)
        reconstructed_edge_image = reconstruct_image(image, edges)
        cv2.imwrite(os.path.join(output_folder, f"{base_filename}_edge_reconstructed.png"), reconstructed_edge_image)

        # PCA-based Reconstructed Image
        reconstructed_pca_image = pca_image_compression(image, n_components)
        cv2.imwrite(os.path.join(output_folder, f"{base_filename}_pca_reconstructed.png"), reconstructed_pca_image)

# Paths
dataset_folder = r"OneDrive - Indian Institute of Technology Bombay/Documents/Data_science_intern/CS663/Weizmann_Seg_DB_1obj/1obj"  # Dataset folder path
output_folder = r"OneDrive - Indian Institute of Technology Bombay/Documents/Data_science_intern/CS663/Output_withPCA"  # Output folder path

# Process all images in the dataset folder and compare methods
# process_images_and_compare(dataset_folder, output_folder, n_components=50)


JPEG compression

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt

# Quantization Matrix
DEFAULT_Q_MATRIX = np.array([
    [16, 11, 10, 16, 24, 40, 51, 61],
    [12, 12, 14, 19, 26, 58, 60, 55],
    [14, 13, 16, 24, 40, 57, 69, 56],
    [14, 17, 22, 29, 51, 87, 80, 62],
    [18, 22, 37, 56, 68, 109, 103, 77],
    [24, 35, 55, 64, 81, 104, 113, 92],
    [49, 64, 78, 87, 103, 121, 120, 101],
    [72, 92, 95, 98, 112, 100, 103, 99],
])

# Compute 2D DCT
def compute_dct(block):
    return cv2.dct(block.astype(np.float32))

# Compute 2D IDCT
def compute_idct(block):
    return cv2.idct(block.astype(np.float32))

# Quantization
def quantize(block, q_matrix):
    return np.round(block / q_matrix)

# Dequantization
def dequantize(block, q_matrix):
    return block * q_matrix

# JPEG Compression
def jpeg_compress(image, q_matrix):
    block_size = 8
    height, width = image.shape
    
    # Pad image to make dimensions divisible by block size
    padded_height = (height + block_size - 1) // block_size * block_size
    padded_width = (width + block_size - 1) // block_size * block_size
    padded_image = np.zeros((padded_height, padded_width), dtype=image.dtype)
    padded_image[:height, :width] = image

    compressed_image = np.zeros_like(padded_image, dtype=np.float32)

    # Process each block
    for i in range(0, padded_height, block_size):
        for j in range(0, padded_width, block_size):
            block = padded_image[i:i+block_size, j:j+block_size]
            dct_block = compute_dct(block)
            quantized_block = quantize(dct_block, q_matrix)
            compressed_image[i:i+block_size, j:j+block_size] = quantized_block

    return compressed_image[:height, :width]

# JPEG Decompression
def jpeg_decompress(compressed_image, q_matrix):
    block_size = 8
    height, width = compressed_image.shape

    # Pad compressed image to make dimensions divisible by block size
    padded_height = (height + block_size - 1) // block_size * block_size
    padded_width = (width + block_size - 1) // block_size * block_size
    padded_compressed_image = np.zeros((padded_height, padded_width), dtype=compressed_image.dtype)
    padded_compressed_image[:height, :width] = compressed_image

    decompressed_image = np.zeros_like(padded_compressed_image, dtype=np.float32)

    # Process each block
    for i in range(0, padded_height, block_size):
        for j in range(0, padded_width, block_size):
            block = padded_compressed_image[i:i+block_size, j:j+block_size]
            dequantized_block = dequantize(block, q_matrix)
            idct_block = compute_idct(dequantized_block)
            decompressed_image[i:i+block_size, j:j+block_size] = idct_block

    return decompressed_image[:height, :width]

# Calculate RMSE
def calculate_rmse(original, decompressed):
    return np.sqrt(np.mean((original - decompressed) ** 2))

# Calculate BPP
def calculate_bpp(compressed_image, original_shape):
    compressed_bits = np.count_nonzero(compressed_image) * 8  # Assuming each non-zero coefficient is stored as a byte
    num_pixels = original_shape[0] * original_shape[1]
    return compressed_bits / num_pixels

# Process Images from Dataset (RMSE-BPP curve for each image)
def process_images_separately(dataset_folder, output_folder, q_matrix, quality_factors):
    os.makedirs(output_folder, exist_ok=True)

    # Fetch all image files in the folder
    image_files = [os.path.join(dp, f) for dp, dn, fn in os.walk(dataset_folder) for f in fn if f.endswith(('.jpg', '.png', '.jpeg', '.bmp', '.tif', '.tiff'))]

    for image_path in image_files:
        # Load image
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        if image is None:
            print(f"Error loading image: {image_path}")
            continue

        rmse_values = []
        bpp_values = []

        for quality in quality_factors:
            scaled_q_matrix = q_matrix * (100 / quality)

            # Compress and decompress
            compressed = jpeg_compress(image, scaled_q_matrix)
            decompressed = jpeg_decompress(compressed, scaled_q_matrix)

            # Calculate RMSE and BPP
            rmse = calculate_rmse(image, decompressed)
            bpp = calculate_bpp(compressed, image.shape)
            rmse_values.append(rmse)
            bpp_values.append(bpp)

        # Plot RMSE vs. BPP for the current image
        plt.plot(bpp_values, rmse_values, marker='o')
        plt.xlabel('Bits Per Pixel (BPP)')
        plt.ylabel('Root Mean Squared Error (RMSE)')
        plt.title(f'RMSE vs. BPP for {os.path.basename(image_path)}')
        plt.grid()

        # Save the plot
        base_filename = os.path.splitext(os.path.basename(image_path))[0]
        plt.savefig(os.path.join(output_folder, f"{base_filename}_rmse_bpp_curve.png"))
        plt.clf()  # Clear figure for next image

# Main Execution
if __name__ == "__main__":
    dataset_folder = r"OneDrive - Indian Institute of Technology Bombay/Documents/Data_science_intern/CS663/Weizmann_Seg_DB_1obj/1obj"
    output_folder = r"OneDrive - Indian Institute of Technology Bombay/Documents/Data_science_intern/CS663/Output_withJPEG"
    
    quality_factors = [10, 20, 30, 40, 50]  # Quality factors to evaluate
    process_images_separately(dataset_folder, output_folder, DEFAULT_Q_MATRIX, quality_factors)
